In [2]:
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import json
import time
import scipy.stats as stats
import numpy as np

mort = "./2012-2016 CardioVasc Mortality Rates.csv"
edu = "./Education.csv"
econo = "./SAIPESNC.csv"

In [3]:
mortality_file = pd.read_csv(mort)
education_file = pd.read_csv(edu)
socio_econo_file = pd.read_csv(econo)

In [4]:
education_file = education_file.rename(columns={'FIPS Code':'County ID'})
education_file.head()

,County ID,State,Area name,"Less than a high school diploma, 2012-2016","High school diploma only, 2012-2016","Some college or associate's degree, 2012-2016","Bachelor's degree or higher, 2012-2016","Percent of adults with less than a high school diploma, 2012-2016","Percent of adults with a high school diploma only, 2012-2016","Percent of adults completing some college or associate's degree, 2012-2016","Percent of adults with a bachelor's degree or higher, 2012-2016"
0,0,US,United States,"27,818,380","58,820,411","62,242,569","64,767,787",13.0,27.5,29.1,30.3
1,1001,AL,Autauga County,"4,528","12,519","10,451","8,968",12.4,34.3,28.7,24.6
2,1003,AL,Baldwin County,"13,956","40,154","44,486","41,350",10.0,28.7,31.8,29.5
3,1005,AL,Barbour County,"4,824","6,422","4,775","2,366",26.2,34.9,26.0,12.9
4,1007,AL,Bibb County,"3,040","6,586","4,234","1,890",19.3,41.8,26.9,12.0


In [5]:
new_edu_columns = ['County ID', 'State', "County Name", '< High School', 'High School', 'Associate',
                  'Bachelors <', 'Perc < High School', 'Perc High School', 'Perc Associate',
                  'Perc Bachelors <']
education_file.columns = new_edu_columns

In [6]:
edu_cleaned_columns = ['County ID', "County Name", 'Perc < High School', 'Perc High School', 'Perc Associate', 'Perc Bachelors <']
education_clean_df = education_file[edu_cleaned_columns]
education_clean_df['MYS'] = ""
education_clean_df.head()

C:\Users\randy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,County ID,County Name,Perc < High School,Perc High School,Perc Associate,Perc Bachelors <,MYS
0,0,United States,13.0,27.5,29.1,30.3,
1,1001,Autauga County,12.4,34.3,28.7,24.6,
2,1003,Baldwin County,10.0,28.7,31.8,29.5,
3,1005,Barbour County,26.2,34.9,26.0,12.9,
4,1007,Bibb County,19.3,41.8,26.9,12.0,


In [7]:
mean_years_schooling = []
for index, row in education_clean_df.iterrows():
    mean_years_schooling.append(((row['Perc < High School']*10) + (row['Perc High School'] * 12) + (row['Perc Associate'] * 14)
                              + (row['Perc Bachelors <'] * 20))/100)
    
education_clean_df['MYS'] = mean_years_schooling
education_clean_df['MYS'].describe(percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

C:\Users\randy\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count    3143.000000
mean       13.989138
std         0.834010
min        12.010000
10%        13.038400
20%        13.280800
30%        13.502000
40%        13.706000
50%        13.882000
60%        14.070000
70%        14.290800
80%        14.603200
90%        15.104800
max        18.602000
Name: MYS, dtype: float64

In [8]:
# Removing county 15005 from our education dataset as well, since it isn't included in our socio_econo file
education_clean_df = education_clean_df[education_clean_df['County ID'] != 15005]
education_clean_df.reset_index().head()

,index,County ID,County Name,Perc < High School,Perc High School,Perc Associate,Perc Bachelors <,MYS
0,0,0,United States,13.0,27.5,29.1,30.3,14.734
1,1,1001,Autauga County,12.4,34.3,28.7,24.6,14.294
2,2,1003,Baldwin County,10.0,28.7,31.8,29.5,14.796
3,3,1005,Barbour County,26.2,34.9,26.0,12.9,13.028
4,4,1007,Bibb County,19.3,41.8,26.9,12.0,13.112


In [9]:
socio_columns = ['State', 'County ID', 'County Name', '1', '2', '3', '4',' 5', '6','7', 'Median Household Income', '8', '9']
socio_econo_file.columns = socio_columns
socio_econo_file.head()

,State,County ID,County Name,1,2,3,4,5,6,7,Median Household Income,8,9
0,0,0,United States,310764925,47240290,46990033,47490547,15.2,15.1,15.3,54134,54052,54216
1,1,1001,Autauga County (AL),54905,7206,5914,8497,13.1,10.8,15.5,53748,49752,57745
2,1,1003,Baldwin County (AL),197091,25850,21888,29811,13.1,11.1,15.1,50976,48035,53917
3,1,1005,Barbour County (AL),23311,6731,5511,7952,28.9,23.6,34.1,32111,29316,34907
4,1,1007,Bibb County (AL),20316,4268,3359,5177,21.0,16.5,25.5,40113,36439,43787


In [10]:
# Change county name to just have county name, not state name, too.
socio_cleaned_columns = ['County ID', 'County Name', 'Median Household Income']
socio_econo_clean_df = socio_econo_file[socio_cleaned_columns]
socio_econo_clean_df.head()

,County ID,County Name,Median Household Income
0,0,United States,54134
1,1001,Autauga County (AL),53748
2,1003,Baldwin County (AL),50976
3,1005,Barbour County (AL),32111
4,1007,Bibb County (AL),40113


In [11]:
# Dropped the fucked row and reset index
socio_econo_clean_df = socio_econo_clean_df[socio_econo_clean_df['County ID'] != 15005]
socio_econo_clean_df.reset_index().head()

,index,County ID,County Name,Median Household Income
0,0,0,United States,54134
1,1,1001,Autauga County (AL),53748
2,2,1003,Baldwin County (AL),50976
3,3,1005,Barbour County (AL),32111
4,4,1007,Bibb County (AL),40113


In [12]:
print(socio_econo_clean_df.iloc[549:555, :])

     County ID           County Name Median Household Income
549      15001    Hawaii County (HI)                   51582
550      15003  Honolulu County (HI)                   74794
552      15007     Kauai County (HI)                   60725
553      15009      Maui County (HI)                   64971
554      16001       Ada County (ID)                   57133
555      16003     Adams County (ID)                   38973


In [13]:
# Now, we can convert Income column to integer
socio_econo_clean_df['Median Household Income'] = pd.to_numeric(socio_econo_clean_df['Median Household Income'], errors='coerce')

In [14]:
socio_econo_clean_df = socio_econo_clean_df.drop_duplicates(['County ID'], keep='first')

In [15]:
print(education_clean_df.shape)
print(socio_econo_clean_df.shape)
# This shows that there are a few extra rows in our education dataset that will need to be addressed, as they will be occupied 
# by empty or NaN values

(3152, 7)
(3145, 3)


In [16]:
# Now, we should be able to merge everything...
education_income_df = pd.merge(education_clean_df, socio_econo_clean_df, on='County ID', how='outer', 
                               suffixes=('_ED', 'SE'), validate='1:1')

education_income_df.head()

,County ID,County Name_ED,Perc < High School,Perc High School,Perc Associate,Perc Bachelors <,MYS,County NameSE,Median Household Income
0,0,United States,13.0,27.5,29.1,30.3,14.734,United States,54134.0
1,1001,Autauga County,12.4,34.3,28.7,24.6,14.294,Autauga County (AL),53748.0
2,1003,Baldwin County,10.0,28.7,31.8,29.5,14.796,Baldwin County (AL),50976.0
3,1005,Barbour County,26.2,34.9,26.0,12.9,13.028,Barbour County (AL),32111.0
4,1007,Bibb County,19.3,41.8,26.9,12.0,13.112,Bibb County (AL),40113.0


In [17]:
education_income_clean_df = education_income_df[['County ID', 'MYS', 
       'County NameSE', 'Median Household Income']]
education_income_clean_df.head()

,County ID,MYS,County NameSE,Median Household Income
0,0,14.734,United States,54134.0
1,1001,14.294,Autauga County (AL),53748.0
2,1003,14.796,Baldwin County (AL),50976.0
3,1005,13.028,Barbour County (AL),32111.0
4,1007,13.112,Bibb County (AL),40113.0


In [18]:
# Move county name to front and rename
education_income_clean_df = education_income_clean_df[['County NameSE', 'County ID',  'MYS', 
       'Median Household Income']]
education_income_clean_df = education_income_clean_df.rename(columns={'County NameSE' : 'County Name (State)'})
education_income_clean_df.head()

,County Name (State),County ID,MYS,Median Household Income
0,United States,0,14.734,54134.0
1,Autauga County (AL),1001,14.294,53748.0
2,Baldwin County (AL),1003,14.796,50976.0
3,Barbour County (AL),1005,13.028,32111.0
4,Bibb County (AL),1007,13.112,40113.0


In [19]:
education_income_clean_df.shape
# This means there are more rows than either of the two original dataframes had initially...  drop those NaN's.

(3154, 4)

In [20]:
education_income_clean_df['County ID'].count()

3154

In [21]:
mortality_clean = mortality_file[['County', 'County Code', 'Crude Rate']]
mortality_clean = mortality_clean.rename(columns={'County Code':'County ID'})
mortality_clean.head()
mortality_clean.shape

(2896, 3)

In [22]:
combined_data = pd.merge(education_income_clean_df, mortality_clean, how='outer', on='County ID', suffixes=('_EI', 'MORT'))

In [23]:
combined_data = combined_data.dropna()
del combined_data['County Name (State)']
combined_data.shape

(2893, 5)

In [24]:
combined_data = combined_data[combined_data['Crude Rate'] != 'Unreliable']
combined_data.shape

(2653, 5)

In [25]:
combined_data['Crude Rate'] = pd.to_numeric(combined_data['Crude Rate'])

In [26]:
combined_data['MYS'].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

count    2653.000000
mean       13.977689
std         0.855245
min        12.010000
10%        13.022400
20%        13.248800
30%        13.449200
40%        13.644000
50%        13.830000
60%        14.036000
70%        14.298000
80%        14.630000
90%        15.152000
max        18.010000
Name: MYS, dtype: float64

In [27]:
combined_data['Education Bin'] = ""

education_bins = combined_data['MYS'].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 
                                                                                    0.8, 0.9])[['min', '10%', '20%', '30%', 
                                                                                                '40%', '50%', '60%', '70%', 
                                                                                                '80%', '90%', 'max']]
education_bin_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
combined_data['Education Bin'] = pd.cut(combined_data['MYS'], bins=education_bins, labels=education_bin_labels)
combined_data.head()

,County ID,MYS,Median Household Income,County,Crude Rate,Education Bin
1,1001,14.294,53748.0,"Autauga County, AL",130.0,7
2,1003,14.796,50976.0,"Baldwin County, AL",113.0,9
3,1005,13.028,32111.0,"Barbour County, AL",165.1,2
4,1007,13.112,40113.0,"Bibb County, AL",195.8,2
5,1009,13.328,46377.0,"Blount County, AL",130.7,3


In [28]:
combined_data['Education Bin'].value_counts().sum()

2652

In [29]:
# We will create the same bins based on these percentile ranges and see if they match up with our education bins
combined_data['Income Bin'] = ""

income_bins = combined_data['Median Household Income'].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 
                                                                                    0.8, 0.9])[['min', '10%', '20%', '30%', 
                                                                                                '40%', '50%', '60%', '70%', 
                                                                                                '80%', '90%', 'max']]
income_bin_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
combined_data['Income Bin'] = pd.cut(combined_data['Median Household Income'], 
                                            bins=income_bins, labels=income_bin_labels)
combined_data.head()

,County ID,MYS,Median Household Income,County,Crude Rate,Education Bin,Income Bin
1,1001,14.294,53748.0,"Autauga County, AL",130.0,7,8
2,1003,14.796,50976.0,"Baldwin County, AL",113.0,9,8
3,1005,13.028,32111.0,"Barbour County, AL",165.1,2,1
4,1007,13.112,40113.0,"Bibb County, AL",195.8,2,3
5,1009,13.328,46377.0,"Blount County, AL",130.7,3,6


In [30]:
combined_data['Income Bin'].value_counts().sum()

2652

In [31]:
combined_data['Mortality Bin'] = ""
mortality_bins = combined_data['Crude Rate'].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 
                                                                                    0.8, 0.9])[['min', '10%', '20%', '30%', 
                                                                                                '40%', '50%', '60%', '70%', 
                                                                                                '80%', '90%', 'max']]
mortality_bin_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
combined_data['Mortality Bin'] = pd.cut(combined_data['Crude Rate'], 
                                            bins=mortality_bins, labels=mortality_bin_labels)
combined_data.head()

,County ID,MYS,Median Household Income,County,Crude Rate,Education Bin,Income Bin,Mortality Bin
1,1001,14.294,53748.0,"Autauga County, AL",130.0,7,8,6
2,1003,14.796,50976.0,"Baldwin County, AL",113.0,9,8,5
3,1005,13.028,32111.0,"Barbour County, AL",165.1,2,1,8
4,1007,13.112,40113.0,"Bibb County, AL",195.8,2,3,9
5,1009,13.328,46377.0,"Blount County, AL",130.7,3,6,7


In [32]:
combined_data['Mortality Bin'].value_counts().sum()

2652

In [33]:
combined_data.head()

,County ID,MYS,Median Household Income,County,Crude Rate,Education Bin,Income Bin,Mortality Bin
1,1001,14.294,53748.0,"Autauga County, AL",130.0,7,8,6
2,1003,14.796,50976.0,"Baldwin County, AL",113.0,9,8,5
3,1005,13.028,32111.0,"Barbour County, AL",165.1,2,1,8
4,1007,13.112,40113.0,"Bibb County, AL",195.8,2,3,9
5,1009,13.328,46377.0,"Blount County, AL",130.7,3,6,7


In [34]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2653 entries, 1 to 3151
Data columns (total 8 columns):
County ID                  2653 non-null int64
MYS                        2653 non-null float64
Median Household Income    2653 non-null float64
County                     2653 non-null object
Crude Rate                 2653 non-null float64
Education Bin              2652 non-null category
Income Bin                 2652 non-null category
Mortality Bin              2652 non-null category
dtypes: category(3), float64(3), int64(1), object(1)
memory usage: 133.3+ KB


In [35]:
combined_data['Income Bin'] = pd.to_numeric(combined_data['Income Bin']).astype(float)
combined_data['Education Bin'] = pd.to_numeric(combined_data['Education Bin']).astype(float)
combined_data['Mortality Bin'] = pd.to_numeric(combined_data['Mortality Bin']).astype(float)
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2653 entries, 1 to 3151
Data columns (total 8 columns):
County ID                  2653 non-null int64
MYS                        2653 non-null float64
Median Household Income    2653 non-null float64
County                     2653 non-null object
Crude Rate                 2653 non-null float64
Education Bin              2652 non-null float64
Income Bin                 2652 non-null float64
Mortality Bin              2652 non-null float64
dtypes: float64(6), int64(1), object(1)
memory usage: 186.5+ KB


In [36]:
combined_data['MYS'].describe(percentiles=[0.1, 0.2, 0.3, .4, .5, .6, .7, .8, .9])

count    2653.000000
mean       13.977689
std         0.855245
min        12.010000
10%        13.022400
20%        13.248800
30%        13.449200
40%        13.644000
50%        13.830000
60%        14.036000
70%        14.298000
80%        14.630000
90%        15.152000
max        18.010000
Name: MYS, dtype: float64

In [37]:
combined_data.shape

(2653, 8)

In [38]:
combined_data = combined_data.dropna()
combined_data.shape

(2650, 8)

In [39]:
combined_data.to_csv('CJK-New-Combined_Dataset.csv', index=False)

In [40]:
from scipy.stats import linregress

In [41]:
stats.linregress(combined_data['Education Bin'], combined_data['Income Bin'])

LinregressResult(slope=0.7151038176237223, intercept=1.5689781380262242, rvalue=0.7149401774756134, pvalue=0.0, stderr=0.013590420522599234)

In [42]:
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff
plotly.tools.set_credentials_file(username='andrewkrog', api_key='srXlzccNxXRUUsUM9uPn')

import numpy as np
import pandas as pd

colorscale = ["#f7fbff","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1",
            "#85bcdb","#6baed6","#3082be","#2171b5","#1361a9",
            "#08519c","#0b4083","#08306b"]
              
endpts = list(np.linspace(0, 125000, len(colorscale) - 1))
fips = combined_data['County ID'].tolist()
values = combined_data['Median Household Income'].tolist()

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['usa'],
    binning_endpoints=endpts, colorscale=colorscale,
    show_state_data=False,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9, title='Median Household Income ($)',
    legend_title='Median Household Income ($)',
)
py.iplot(fig, filename='choropleth_full_usa')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~andrewkrog/0 or inside your plot.ly account where it is named 'choropleth_full_usa'


C:\Users\randy\AppData\Local\Continuum\anaconda3\lib\site-packages\plotly\api\v1\clientresp.py:40: UserWarning:

Estimated Draw Time Slow



In [43]:
colorscale = ["#f7ffe6","#c4ff4d","#b3ff1a","#aaff00",
            "#99e600","#77b300","#669900","#558000","#334d00",
            "#223300"]
              
endpts = list(np.linspace(8, 18, len(colorscale) - 1))
fips = combined_data['County ID'].tolist()
values = combined_data['MYS'].tolist()

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['usa'],
    binning_endpoints=endpts, colorscale=colorscale,
    show_state_data=False,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9, title='Mean Year Schooling',
    legend_title='Mean Year Schooling',
)
py.iplot(fig, filename='choropleth_full_usa')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~andrewkrog/0 or inside your plot.ly account where it is named 'choropleth_full_usa'


C:\Users\randy\AppData\Local\Continuum\anaconda3\lib\site-packages\plotly\api\v1\clientresp.py:40: UserWarning:

Estimated Draw Time Slow



In [44]:
colorscale = ["#ffebe6","#ffd6cc","#ffad99","#ff8566",
            "#ff5c33","#ff471a","#ff3300","#991f00","#801a00",
            "#661400"]
              
endpts = list(np.linspace(0, 500, len(colorscale) - 1))
fips = combined_data['County ID'].tolist()
values = combined_data['Crude Rate'].tolist()

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['usa'],
    binning_endpoints=endpts, colorscale=colorscale,
    show_state_data=False,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9, title='Crude Rate',
    legend_title='Crude Rate',
)
py.iplot(fig, filename='choropleth_full_usa')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~andrewkrog/0 or inside your plot.ly account where it is named 'choropleth_full_usa'


C:\Users\randy\AppData\Local\Continuum\anaconda3\lib\site-packages\plotly\api\v1\clientresp.py:40: UserWarning:

Estimated Draw Time Slow



In [45]:
import joypy

combined_data["Count"] = combined_data.groupby('Income Bin').cumcount() + 1
combined_data_pivot = combined_data[['Income Bin', 'Count', 'Crude Rate']].pivot(index = 'Count', columns = 'Income Bin')
combined_data_pivot = pd.DataFrame(combined_data_pivot.to_records())
combined_data_pivot.columns = ['Count', 'NaN','0-10th Percentile', '20th', '30th', '40th', '50th','60th', '70th', '80th', '90th', '100th']
del combined_data_pivot['Count']
del combined_data_pivot['NaN']

fig, axes = joypy.joyplot(combined_data_pivot,
                          title = 'Distribution of Heart Disease Deaths per 100,000\nby County Median Income Decile',
                          overlap = 0.75,
                          figsize = (7,5)
                         )

ValueError: Length mismatch: Expected axis has 11 elements, new values have 12 elements

In [51]:
combined_data["Count"] = combined_data.groupby('Education Bin').cumcount() + 1
combined_data_pivot_educ = combined_data[['Education Bin', 'Count', 'Crude Rate']].pivot(index = 'Count', columns = 'Education Bin')
combined_data_educ = pd.DataFrame(combined_data_pivot_educ.to_records())
combined_data_pivot_educ.columns = ['Count', 'NaN',
                               '0-10th Percentile', '10-20th', '20-30th', '30-40th', '40-50th',
                               '50-60th', '60-70th', '70-80th', '80-90th', '90-100th']
del combined_data_pivot_educ['Count']
del combined_data_pivot_educ['NaN']

fig, axes = joypy.joyplot(combined_data_pivot_educ,
                          title = 'Distribution of Heart Disease Deaths per 100,000\nby County Education Decile',
                          overlap = 0.75,
                          figsize = (7,5)
                         )

ValueError: Length mismatch: Expected axis has 10 elements, new values have 12 elements